In [1]:
from carsus import init_db
session = init_db("sqlite://")
session.commit()

Initializing the database
Ingesting basic atomic data


In [2]:
from carsus.io.nist import NISTWeightsCompIngester
ingester = NISTWeightsCompIngester()
ingester.download()
ingester.ingest(session)

Ingesting atomic weights


In [3]:
from carsus.io.output import AtomsDataset, Dataset

# Create a dataset and load with the atoms data from the database. 

atoms = AtomsDataset()
atoms.load_sql(session)
print atoms.data.head(5)


              symbol       name  group  period
atomic_number                                 
1                  H   Hydrogen      1       1
2                 He     Helium     18       1
3                 Li    Lithium      1       2
4                 Be  Beryllium      2       2
5                  B      Boron     13       2


In [4]:
# Include atomic weights into the dataset

atoms2 = AtomsDataset()
atoms2.load_sql(session, load_atomic_weights=True)
print atoms2.data.head(5)

              symbol       name  group  period  atomic_weight_value  \
atomic_number                                                         
1                  H   Hydrogen      1       1             1.007975   
2                 He     Helium     18       1             4.002602   
3                 Li    Lithium      1       2             6.967500   
4                 Be  Beryllium      2       2             9.012183   
5                  B      Boron     13       2            10.813500   

               atomic_weight_uncert atomic_weight_unit  \
atomic_number                                            
1                      1.350000e-04                  u   
2                      2.000000e-06                  u   
3                      2.950000e-02                  u   
4                      5.000000e-07                  u   
5                      7.500000e-03                  u   

              atomic_weight_data_source  
atomic_number                            
1          

In [5]:
# It is easy to create new computed fields with pandas dataframes. 
# Special methods of the classes derived from the ``Dataset`` class could be used to create such fields

atoms2.data["computed_field"] = atoms2.data["atomic_weight_value"]**2 + 10
print atoms2.data["computed_field"].head(5)

atomic_number
1     11.016014
2     26.020823
3     58.546056
4     91.219444
5    126.931782
Name: computed_field, dtype: float64


In [6]:
# It is also possible to create a dataset manually

from numpy.random import randn
from pandas import DataFrame
df = DataFrame(randn(8, 3), columns=['A', 'B', 'C'])
some_dataset = Dataset(name="some", data=df)

In [7]:
# create_hdf() is used to put several datasets into an HDF file. Datasets' names are used as the identifiers in the file.

from carsus.io.output import create_hdf
create_hdf("store.h5", datasets=[atoms, some_dataset])


/home/misha/.virtualenvs/carsus/lib/python2.7/site-packages/ipykernel/__main__.py:2: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->unicode,key->block1_values] [items->['symbol', 'name']]

  from ipykernel import kernelapp as app


In [8]:
from pandas import read_hdf
atoms = read_hdf("store.h5", "atoms")
print atoms.head(5)

              symbol       name  group  period
atomic_number                                 
1                  H   Hydrogen      1       1
2                 He     Helium     18       1
3                 Li    Lithium      1       2
4                 Be  Beryllium      2       2
5                  B      Boron     13       2
